## Generate Shell File

In [1]:
def topoinf_train_model_wrapper(
        py_file = 'topoinf_pseudo_label_train_model.py',

        model = 'GCN',
        dataset = 'cora',
        num_layers = 3,

        delete_unit = 'number',
        delete_mode = 'pos',
        delete_rate_list = [0.1],
        delete_num_list = [400],
        topoinf_threshold = None,

        one_hot_mask = ['train_mask', 'val_mask'],
        correct_and_smooth = False,
    ):
    command = ''

    # model & dataset
    command += f"\npython {py_file} \\"   # NOTE: .py file
    command += f"\n\t--model {model} \\"
    command += f"\n\t--dataset {dataset} \\"
    command += f"\n\t--num-layers {num_layers} \\"
    
    # topoinf setting
    command += f"\n\t--delete-unit {delete_unit} \\"
    command += f"\n\t--delete-mode {delete_mode} \\"
    command += f"\n\t--delete-rate-list {str(delete_rate_list).strip('[]').replace(', ', ' ')} \\" if delete_unit in ['mode_ratio',  'ratio'] \
                else f"\n\t--delete-num-list {str(delete_num_list).strip('[]').replace(', ', ' ')} \\"
    
    command += f"\n\t--one-hot-mask {' '.join(one_hot_mask)} \\"
    if correct_and_smooth:
        command += f"\n\t--use-correct-and-smooth \\"
        
    command += f"\n\t--topoinf-threshold {topoinf_threshold} \\" if topoinf_threshold is not None else ''
    
    command += '\n\n'
    
    return command

# test_string = topoinf_train_model_wrapper()
# print(test_string)

In [2]:
DATASETS = ['cora', 'citeseer', 'pubmed']
MODELS = ['GCN', 'SGC', 'APPNP']
## TODO: GPRGNN, BernNet, ...
# CORRECT_AND_SMOOTHES = [True, False]
CORRECT_AND_SMOOTHES = [False]
ONE_HOT_MASKS = [['train_mask'], ['train_mask', 'val_mask']]

In [3]:
exp_1_run_sh = ''
exp_1_run_sh += '### `exp_1_run.sh` File ###'
exp_1_run_sh += '\n### TO RUN: `sh exp_1_run.sh` ###\n'

# exp_1_run_sh += '\n' + \
# """
# ## activate conda env
# eval "$(conda shell.bash hook)"
# conda activate pyg
# """

print(exp_1_run_sh)

### `exp_1_run.sh` File ###
### TO RUN: `sh exp_1_run.sh` ###



In [4]:
task_counter = 0
for dataset in DATASETS:
    # if dataset in ['pubmed']:
    #     topoinf_threshold = 1e-3    # NOTE: TopoInf in large graphs will be relatively smaller.
    # else:
    #     topoinf_threshold = 5e-2
    
    if dataset in ['pubmed']:
        delete_num_list = [500, 1000, 1500, 2000, 2500, 3000]
    else:
        delete_num_list = [100, 200, 300, 400, 500, 600]
    
    for model in MODELS:

        for correct_and_smooth in CORRECT_AND_SMOOTHES:
            for one_hot_mask in ONE_HOT_MASKS:
                exp_1_run_sh += \
                    topoinf_train_model_wrapper(dataset=dataset,
                                                model=model,
                                                delete_mode='pos',
                                                delete_unit='number',
                                                
                                                correct_and_smooth=correct_and_smooth,
                                                one_hot_mask=one_hot_mask,
                                                delete_num_list=delete_num_list,

                                                # topoinf_threshold=topoinf_threshold,
                                                )
                task_counter += 1

In [5]:
exp_1_run_sh += \
f"""
###################################
#      NUM TOTAL TASKS: {task_counter:4d}      #
###################################
"""

exp_1_run_sh_file_path = './run_exp.sh'
with open(exp_1_run_sh_file_path, 'w') as f:
    f.write(exp_1_run_sh)